In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import collect_list, concat_ws, udf ,lit, col, when, split, size, lower, explode, dayofmonth
from pyspark.sql.types import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, when, lit, avg, stddev, month, year, unix_timestamp, current_date, date_sub
from pyspark.sql.functions import col, max, min, lit
from pyspark.sql.functions import date_add, to_date
import pandas as pd
import plotly.express as px
from urllib.request import urlopen
import json
import plotly.graph_objects as go

In [0]:
df_original = spark.table("marketingdata_prod.ds_sandbox.prediction_mean_yw")

In [0]:
df_3 = spark.table("marketingdata_prod.ds_sandbox.df_3_plotly_yw")

In [0]:
df_3_pd = df_3.toPandas()

In [0]:
#df_original.display()

In [0]:
import json
with open("/dbfs/FileStore/contact_centre/FileShare/uk_postcode_sectors_2.geojson", "r") as f: postcode_sectors_geojson_2 = json.load(f)

In [0]:
fig = px.choropleth_mapbox(
    df_3_pd,
    geojson=postcode_sectors_geojson_2,
    locations="postcode_3",
    color="prediction",
    color_continuous_scale="Viridis",
    range_color=(0, 1),
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 54.5, "lon": -2},
    opacity=0.5,
    #labels={"unemp": "unemployment rate"},
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
#fig.show()

In [0]:
fig.write_html("/dbfs/FileStore/contact_centre/FileShare/fig_choropleth_yw.html")

## 2. Branch Scatter

In [0]:
branch_coor = spark.table("marketingdata_prod.ds_sandbox.branch_for_scatter_yw")

In [0]:
#branch_coor.display()

In [0]:
branch_coor_pd = branch_coor.toPandas()

In [0]:
overlay_map = px.choropleth_mapbox(
    df_3_pd,
    geojson=postcode_sectors_geojson_2,
    locations="postcode_3",
    color="prediction",
    color_continuous_scale="Viridis",
    range_color=(0, 1),
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 54.5, "lon": -2},
    opacity=0.3,
)

overlay_map.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

branch = go.Scattermapbox(
    lat=branch_coor_pd['Latitude'], 
    lon=branch_coor_pd['Longitude'],  
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=7,
        color='black', 
        opacity=0.7
    ),
    text=branch_coor_pd['branch_code_desc'], 
)

#overlay_map = go.overlay_mapure(overlay_map)
overlay_map.add_trace(branch)

overlay_map.write_html("/dbfs/FileStore/contact_centre/FileShare/map_overlay_yw.html")